In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# save filepath 
home_data_file_path = '/Users/balqeesjabri/Downloads/train.csv'
# read the data and store data in DataFrame home_data
home_data = pd.read_csv(home_data_file_path)

In [ ]:
# target
y = home_data.SalePrice
X = home_data.drop(['SalePrice'], axis=1)
# Divide data into training and validation subsets
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y,
                                                                random_state=0)

In [ ]:
# Select categorical columns 
categorical_cols = [colname for colname in X_train_full.columns if X_train_full[colname].dtype == "object"]

# Select numerical columns
numerical_cols = [colname for colname in X_train_full.columns if X_train_full[colname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.decomposition import PCA
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np


#Create transformers for numerical features 
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler()),
   
])

# Create transformers for categorical features
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
])

# Create a ColumnTransformer to apply different transformers to both feature types  feature types
preprocessor = ColumnTransformer(transformers=[
    ('num', numerical_transformer, numerical_cols),
    ('cat', categorical_transformer, categorical_cols)
])


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.ensemble import RandomForestRegressor
# create a pipline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', RandomForestRegressor(n_estimators=250,
                                                              random_state=55))
                             ])
# This line fits the pipeline to the training data,
my_pipeline.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_valid)

# Evaluate the model
score = mean_absolute_error(y_valid, preds)
rmse = np.sqrt(mean_squared_error(y_valid, preds))
print('rmse:', rmse)

In [ ]:
# Preprocess the data using the preprocessor
X_train_preprocessed = preprocessor.transform(X_train)
X_valid_preprocessed = preprocessor.transform(X_valid)

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=None)
pca.fit(X_train_preprocessed)

exp_var = pca.explained_variance_ratio_ * 100
cum_exp_var = np.cumsum(exp_var)
plt.figure(figsize=(15, 10))
plt.bar(range(1, 81), exp_var, align='center',
        label='Individual explained variance', width=0.5)
plt.step(range(1, 81), cum_exp_var, where='mid',
         label='Cumulative explained variance', color='red')

plt.ylabel('Explained variance percentage')
plt.xlabel('Principal component index')
plt.xticks(ticks=list(range(1, 81)))
plt.legend(loc='best')
plt.tight_layout()

plt.savefig("cumulative_explained_variance_plot.png")

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=49)
X_pca = pd.DataFrame(pca.fit_transform(X_train_preprocessed)
)


In [ ]:
# from sklearn.model_selection import cross_val_score
# # the number of folds here is 5

# scores = -1 * cross_val_score(my_pipeline, X, y,
#                               cv=5,
#                               scoring='neg_mean_absolute_error')

# # Calculate the average cross-validation RMSE score
# avg_cross_val_rmse = np.sqrt(np.mean(scores**2))

# print("RMSE scores:\n", avg_cross_val_rmse)
# print("Average MAE score (across experiments):")
# print(scores.mean())

In [12]:
import numpy as np
from sklearn.model_selection import cross_val_score

# The number of folds here is 5
scores = -1 * cross_val_score(my_pipeline, X, y,
                              cv=5,
                              scoring='neg_mean_squared_error')

# Calculate the average cross-validation RMSE score
avg_cross_val_rmse = np.sqrt(np.mean(scores))

print("RMSE scores:\n", avg_cross_val_rmse)
print("Average MAE score (across experiments):")
print(scores.mean())


RMSE scores:
 29923.87843973819
Average MAE score (across experiments):
895438500.876228
